### Lab: Celery
* create a Celery task named __`write_file()`__ which takes two arguments, a filename and a string, and write the string to the file
* create a second task named __`read_file()`__ which reads from a file
  * if the file does not exist, it waits until the file exists (perhaps have it poll once a second)
  * after a successful read, the task should delete the file
* test your tasks
* incorporate the task into a program which gets input from the user, and uses the tasks above to read and write data to a file (imagine that the files are on a remote system, even though that won't be the case for our lab)

In [ ]:
cd data

In [9]:
%%file celery_examples/file_tasks.py
import os
import time

from celery import Celery

app = Celery('tasks', backend='rpc://', broker='amqp://guest@localhost')

@app.task
def write_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

@app.task
def read_file(filename):
    while not os.path.exists(filename):
        time.sleep(1)
    with open(filename) as f:
        data = f.read()
    os.remove(filename)
    return data

Overwriting celery_examples/file_tasks.py


In [10]:
from celery_examples.file_tasks import write_file, read_file

filename = input('Filename? ')
content = input('Content? ')

write_file.delay(filename, content)
result = read_file.delay(filename)
print(result.get())


Filename? /tmp/foobar.txt
Content? DOes this work?
DOes this work?
